In [1]:
!pip install folium
import pandas as pd
import folium
import requests

In [2]:
CLIENT_ID = 'K2MWVJRUBI3XD2ITBIEW5IIO5DJMT2ZNH5TTV1NY53G5CFL3'
CLIENT_SECRET = 'C5M1BNRTWULRGCADCR5TIMUBCVAKUOWR30YZZYLUXULPSUM1'

In [3]:
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Helsinki",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [5]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [6]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Helsinki', 'Helsinki')

In [7]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 60.178172775811575, 'lng': 24.952162607605953},
  'sw': {'lat': 60.160364637231424, 'lng': 24.921636447256862}},
 86)

In [8]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [9]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '50f688d5e4b023d2f274b506',
  'name': 'Kaffecentralen',
  'location': {'address': 'Fredrikinkatu 59',
   'crossStreet': 'Kansakoulukatu',
   'lat': 60.167580051384675,
   'lng': 24.932525558737044,
   'labeledLatLngs': [{'label': 'display',
     'lat': 60.167580051384675,
     'lng': 24.932525558737044}],
   'postalCode': '00100',
   'cc': 'FI',
   'city': 'Helsinki',
   'state': 'Uusimaa',
   'country': 'Suomi',
   'formattedAddress': ['Fredrikinkatu 59 (Kansakoulukatu)',
    '00100 Helsinki',
    'Suomi']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referra

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 48 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,50f688d5e4b023d2f274b506,Kaffecentralen,Coffee Shop,Fredrikinkatu 59,00100,60.167580,24.932526
1,5aec747112f0a9002c9b92ab,La Torrefazione,Café,Mannerheimintie 22,00100,60.170721,24.936158
2,4b4cb879f964a520c0bb26e3,The Ounce,Tea Room,Fredrikinkatu 55,00100,60.167182,24.932993
3,556f2874498e103ac120a121,Kissakahvila Helkatti,Pet Café,Fredrikinkatu 55,00100,60.167274,24.933142
4,5b10f0fe7269fe002cec8df8,St. George Bakery,Bakery,Yrjönkatu 13,00120,60.166676,24.940574


In [12]:
helsinki_center = d["geocode"]["center"]
helsinki_center

{'lat': 60.16952, 'lng': 24.93545}

In [13]:
from folium import plugins

# create map of Helsinki using latitude and longitude values
map_helsinki = folium.Map(location=[helsinki_center["lat"], helsinki_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_helsinki)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_helsinki.add_child(plugins.HeatMap(hm_data))

map_helsinki

In [14]:
lat = 60.168749
lng = 24.945747
map_helsinki = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_helsinki)
map_helsinki